# Lab 7

Ali Tejani, amt3639

Caroline Yao, chy253

Allen Hwang, 45755

## Problem 3.  
(Starting with MNIST) Install Tensorflow and Keras 2.0.  Use the amazon instancesand complete this tutorial:https://www.tensorflow.org/get_started/mnist/pros

### Load MNIST Data

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Start TensorFlow InteractiveSession

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

### Build a Softmax Regression Model

Placeholders

In [3]:
# create nodes for input and output
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

Variables

In [4]:
# weights of inputs
W = tf.Variable(tf.zeros([784,10]))
# biases of outputs
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

Predicted Class and Loss Function

In [5]:
# implement regression model
y = tf.matmul(x,W) + b

# loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

### Train the Model

In [6]:
# train using Gradient Descent on cross entropy with step of 0.5
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [7]:
# repeatedly run train_step using 100 new samples at a time, replacing the placeholders created earlier
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

Evaluate the Model

In [8]:
# compare prediction to actual
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [10]:
# compute accuracy of model
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9157


### Build a Multilayer Convolutional Network

Weight Initialization

In [11]:
# add noise to weight
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# give slightly positive biase to avoid dead neurons
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

Convolution and Pooling

In [12]:
# convolve with stride of 1 and zero padding
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# max pooling over 2x2 blocks
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

First Convolutional Layer

In [13]:
# for each 5x5 patch with 1 input channel, compute 32 features
W_conv1 = weight_variable([5, 5, 1, 32])
# component for each output channel
b_conv1 = bias_variable([32])

# 28x28 image with one color channel
x_image = tf.reshape(x, [-1, 28, 28, 1])

# convolve with weight tensor, add bias, apply relu function, and max pool over 2x2 blocks
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

Second Convolutional Layer

In [14]:
# convert 32 features from last layer to 64 features
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Densely Connected Layer

In [15]:
# 7x7 image with 64 features, 1024 neurons
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# reshape into batch of vectors, multiply by weight, add bias, apply relu
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Dropout

In [16]:
# apply dropout to avoid overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Readout Layer

In [17]:
# apply softmax regression
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

Train and Evaluate the Model

In [18]:
# loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
# Use ADAM instead of Gradient Descent
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# Compare predictions to actual
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
# Accuracy function
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        # train with 50 samples at a time
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            # log accuracy
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        # train, replace x and y placeholders with samples, and keep probability with 0.5
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    # print total accuracy
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.88
step 200, training accuracy 0.92
step 300, training accuracy 0.86
step 400, training accuracy 0.88
step 500, training accuracy 0.96
step 600, training accuracy 0.96
step 700, training accuracy 0.9
step 800, training accuracy 0.92
step 900, training accuracy 0.94
step 1000, training accuracy 0.94
step 1100, training accuracy 0.94
step 1200, training accuracy 1
step 1300, training accuracy 0.96
step 1400, training accuracy 1
step 1500, training accuracy 0.98
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.94
step 1900, training accuracy 0.96
step 2000, training accuracy 0.96
step 2100, training accuracy 0.98
step 2200, training accuracy 0.96
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 0.94
step 2800, training accuracy 0.98
step 2900, training accuracy 1
step 3000, trainin